In [13]:
# 初回だけ実行
%pip install pandas
%pip install tabulate
# %pip install rp2040-pio-emulator@git+https://github.com/NathanY3G/rp2040-pio-emulator
# %pip install adafruit-circuitpython-pioasm

     ---------------------------------------- 0.0/61.0 kB ? eta -:--:--
     ------ --------------------------------- 10.2/61.0 kB ? eta -:--:--
     ---------------------------------------- 61.0/61.0 kB 1.1 MB/s eta 0:00:00
   ---------------------------------------- 0.0/11.5 MB ? eta -:--:--
   ---- ----------------------------------- 1.2/11.5 MB 36.5 MB/s eta 0:00:01
   ---------- ----------------------------- 3.0/11.5 MB 38.7 MB/s eta 0:00:01
   ----------------- ---------------------- 4.9/11.5 MB 44.6 MB/s eta 0:00:01
   ----------------------- ---------------- 6.8/11.5 MB 48.1 MB/s eta 0:00:01
   ----------------------------- ---------- 8.5/11.5 MB 49.1 MB/s eta 0:00:01
   ----------------------------------- ---- 10.1/11.5 MB 45.9 MB/s eta 0:00:01
   ---------------------------------------  11.5/11.5 MB 50.4 MB/s eta 0:00:01
   ---------------------------------------- 11.5/11.5 MB 46.7 MB/s eta 0:00:00
   ---------------------------------------- 0.0/15.5 MB ? eta -:--:--
   -----

In [24]:
import pandas as pd
import tabulate
import itertools
import adafruit_pioasm
import pioemu

# NAND IC Interconnect
# | 31  | 30  | 29  | 28  | 27  | 26  | 25  | 24  | 23  | 22  | 21  | 20  | 19  | 18  | 17  | 16  | 15  | 14  | 13  | 12  | 11  | 10  | 9    | 8    | 7   | 6   | 5   | 4   | 3   | 2   | 1   | 0   |
# | --- | --- | --- | --- | --- | --- | --- | --- | --- | --- | --- | --- | --- | --- | --- | --- | --- | --- | --- | --- | --- | --- | ---- | ---- | --- | --- | --- | --- | --- | --- | --- | --- |
# | --  | --  | --  | --  | --  | --  | led | --  | --  | --  | --  | --  | --  | --  | --  | --  | rbb | reb | web | wpb | ale | cle | ceb1 | ceb0 | io7 | io6 | io5 | io4 | io3 | io2 | io1 | io0 |

program = """
.program pio-nandio-readid
    set x 9
loop:
    jmp x-- loop
    wait 1 gpio 0
"""
opcodes = adafruit_pioasm.assemble(program)


# emulation 
test_cycles = 100
generator = pioemu.emulate(
    opcodes=opcodes, stop_when=lambda _, state: state.clock > test_cycles
)

# run
results = [after.__dict__ for before, after in itertools.islice(generator, test_cycles)]
results_df = pd.DataFrame.from_records(results)

# 後処理
results_df["program_inst"] = results_df["program_counter"].map(lambda pc: opcodes[pc])
results_df["program_inst_hex"] = results_df["program_inst"].map(lambda opcode: hex(opcode)) # 余裕あったらDecodeしてもいいかも

results_df["io"] = results_df["pin_values"] & 0x000000FF
results_df["io_hex"] = results_df["io"].map(lambda data: hex(data))

results_df["ceb0"] = results_df["pin_values"] & 0x00000100
results_df["ceb1"] = results_df["pin_values"] & 0x00000200
results_df["cle"] = results_df["pin_values"] & 0x00000400
results_df["ale"] = results_df["pin_values"] & 0x00000800
results_df["wpb"] = results_df["pin_values"] & 0x00001000
results_df["web"] = results_df["pin_values"] & 0x00002000
results_df["reb"] = results_df["pin_values"] & 0x00004000
results_df["rbb"] = results_df["pin_values"] & 0x00008000
results_df["led"] = results_df["pin_values"] & 0x01000000

pd.set_option('display.max_rows', 100)
pd.set_option('display.max_columns', 100)
results_df


,clock,program_counter,pin_directions,pin_values,transmit_fifo,receive_fifo,input_shift_register,output_shift_register,x_register,y_register,program_inst,program_inst_hex,io,io_hex,ceb0,ceb1,cle,ale,wpb,web,reb,rbb,led
0,1,1,0,0,[],[],"ShiftRegister(contents=0, counter=0)","ShiftRegister(contents=0, counter=32)",9,0,65,0x41,0,0x0,0,0,0,0,0,0,0,0,0
1,2,1,0,0,[],[],"ShiftRegister(contents=0, counter=0)","ShiftRegister(contents=0, counter=32)",8,0,65,0x41,0,0x0,0,0,0,0,0,0,0,0,0
2,3,1,0,0,[],[],"ShiftRegister(contents=0, counter=0)","ShiftRegister(contents=0, counter=32)",7,0,65,0x41,0,0x0,0,0,0,0,0,0,0,0,0
3,4,1,0,0,[],[],"ShiftRegister(contents=0, counter=0)","ShiftRegister(contents=0, counter=32)",6,0,65,0x41,0,0x0,0,0,0,0,0,0,0,0,0
4,5,1,0,0,[],[],"ShiftRegister(contents=0, counter=0)","ShiftRegister(contents=0, counter=32)",5,0,65,0x41,0,0x0,0,0,0,0,0,0,0,0,0
5,6,1,0,0,[],[],"ShiftRegister(contents=0, counter=0)","ShiftRegister(contents=0, counter=32)",4,0,65,0x41,0,0x0,0,0,0,0,0,0,0,0,0
6,7,1,0,0,[],[],"ShiftRegister(contents=0, counter=0)","ShiftRegister(contents=0, counter=32)",3,0,65,0x41,0,0x0,0,0,0,0,0,0,0,0,0
7,8,1,0,0,[],[],"ShiftRegister(contents=0, counter=0)","ShiftRegister(contents=0, counter=32)",2,0,65,0x41,0,0x0,0,0,0,0,0,0,0,0,0
8,9,1,0,0,[],[],"ShiftRegister(contents=0, counter=0)","ShiftRegister(contents=0, counter=32)",1,0,65,0x41,0,0x0,0,0,0,0,0,0,0,0,0
9,10,1,0,0,[],[],"ShiftRegister(contents=0, counter=0)","ShiftRegister(contents=0, counter=32)",0,0,65,0x41,0,0x0,0,0,0,0,0,0,0,0,0
